## Experiments of Linear Attention

In [1]:
#@title Import internal sources (from github)
from src.transformer import Transformer
from src.data import create_weights
from src.config import config
from src.train import *

In [2]:
# configuration
num_seeds = 1
num_layers = 1

config.num_seeds = num_seeds
config.analyse = True
config.seed = 0

config.in_proj = False
config.adam = True
config.dataset_size = 10
config.input_size = 10
config.num_layers = num_layers
config.input_range = 1


if config.num_layers == 1:
  assert config.deq == True
  assert config.gd_deq == True

if config.num_layers > 1:
  assert config.y_update == False

config.distract_size = 0
config.training_steps_gd = 5#1000 if config.gd_deq else 30000  

In [3]:
#@title Lists

loss_trans_list =  [[]  for _ in range(config.num_seeds)]
loss_trans_train_list =  [[]  for _ in range(config.num_seeds)]
losses_gd_list =  [[]  for _ in range(config.num_seeds)]
losses_gd_list_trained =  [[]  for _ in range(config.num_seeds)]
losses_int_list_trained =  [[]  for _ in range(config.num_seeds)]
cos_sim_list, cos_sim_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]
grad_norm_list, grad_norm_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]
p_norm_list, p_norm_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]

cos_sim_list, cos_sim_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]
grad_norm_list, grad_norm_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]
p_norm_list, p_norm_list_o =  [[]  for _ in range(config.num_seeds)],  [[]  for _ in range(config.num_seeds)]

ir_t_list = [[]  for _ in range(config.num_seeds)]
ws_t_list = [[]  for _ in range(config.num_seeds)]
ir_gd_list = [[]  for _ in range(config.num_seeds)]
ws_gd_list = [[]  for _ in range(config.num_seeds)]

ir_t_ood_list = [[]  for _ in range(config.num_seeds)]
ws_t_ood_list = [[]  for _ in range(config.num_seeds)]
ir_gd_ood_list = [[]  for _ in range(config.num_seeds)]
ws_gd_ood_list = [[]  for _ in range(config.num_seeds)]

ir_gd_trained_list = [[]  for _ in range(config.num_seeds)]
ws_gd_trained_list = [[]  for _ in range(config.num_seeds)]
ir_gd_ood_trained_list = [[]  for _ in range(config.num_seeds)]
ws_gd_ood_trained_list = [[]  for _ in range(config.num_seeds)]

ir_inter_list = [[]  for _ in range(config.num_seeds)]
ws_inter_list = [[]  for _ in range(config.num_seeds)]
ir_inter_ood_list = [[]  for _ in range(config.num_seeds)]
ws_inter_ood_list = [[]  for _ in range(config.num_seeds)]


losses_noisy_list = [[]  for _ in range(config.num_seeds)]
losses_gd_noisy_list = [[]  for _ in range(config.num_seeds)]
losses_gd_noisy_trained_list = [[]  for _ in range(config.num_seeds)]
losses_inter_noisy_list = [[]  for _ in range(config.num_seeds)]

In [4]:
#@title Logic how to interpolate weights
def interpolate_weights(train_state, params_gd, deq=False):
  if (config.num_heads == 1 and 
      config.sum_norm == False and config.deq == True and
      config.layer_norm == False and config.att_only_trans == True):

    cur_train_params = {k.replace('transformer', 'Transformer_gd'):v.copy() for 
                    k,v in train_state.params.items()}

    inter_params = {k.replace('transformer', 'Transformer_gd'): {'w': jnp.zeros_like(v['w'])} for 
                    k,v in train_state.params.items()}

    for k,v in cur_train_params.items():
      if "key" in k:
        key_gd  = params_gd[k]['w'].copy() 
        key  = cur_train_params[k]['w'].copy()
      if "linear" in k:
        linear_gd = params_gd[k]['w'].copy() 
        linear = cur_train_params[k]['w'].copy() 
      if "query" in k:
        query_gd = params_gd[k]['w'].copy() 
        query = cur_train_params[k]['w'].copy() 
      if "value" in k:
        value_gd = params_gd[k]['w'].copy() 
        value = cur_train_params[k]['w'].copy()        
        
        query = jnp.matmul(query, key.T)
        #print(query)
        key = jnp.identity(query.shape[0])
        mean = np.mean([query[a, a] for a in range(query.shape[0]-1)])
        query = query/mean
        query_gd = jnp.matmul(query_gd, key.T)
        key_gd = jnp.identity(query.shape[0])
        query = (query + query_gd)/2

        linear = jnp.matmul(value, linear)
        #print(linear)
        value = jnp.identity(query.shape[0])
        linear = linear*mean
        linear_gd = jnp.matmul(value_gd, linear_gd)
        value_gd = jnp.identity(query.shape[0])
        linear = (linear + linear_gd)/2    

        inter_params[k.replace('value', 'linear')]['w'] = linear
        inter_params[k.replace('value', 'value')]['w'] = value
        inter_params[k.replace('value', 'query')]['w'] = query
        inter_params[k.replace('value', 'key')]['w'] = key

    losses_int, _, _ = predict_test.apply(inter_params, eval_rng, eval_data, True)
  else:
    losses_int = None
    inter_params = None
  return losses_int, inter_params

In [5]:
#@title Create datasets

"""Data utils.
  Provide functions to create regression datasets.
"""

from functools import partial
import jax
from jax import vmap
import jax.numpy as jnp
import numpy as np


@partial(jax.jit, static_argnums=(1, 2, 3))
def create_data_normal(rng, i_size, c_size, size_distract, input_range, w_scale):
  """Create a linear regression data set: X*w where x ~ N(0, 1), w ~ N(1,1)."""

  rng, new_rng, new_rng2, new_rng3, new_rng4 = jax.random.split(rng, 5)
  w = 1 + jax.random.normal(rng, shape=[i_size])*w_scale
  x = jax.random.normal(new_rng, shape=[c_size, i_size])*w_scale
  x_querry = jax.random.normal(new_rng2, shape=[1, i_size])*w_scale

  y_data = jnp.squeeze(x@w)

  y_target = x_querry@w
  y_target = y_target[..., None]


  seq = jnp.concatenate([x, y_data[..., None]], -1)
  target = jnp.concatenate([x_querry, y_target], -1)
  x_querry_init = -1*x_querry.dot(jnp.ones_like(x_querry).T*0.0)
  zero = jnp.concatenate([x_querry, x_querry_init], -1)
  seq = jnp.concatenate([seq, zero], 0)
  return jnp.squeeze(seq), jnp.squeeze(target), w



@partial(jax.jit, static_argnums=(1, 2, 3))
def create_data_normal_noisy(rng, i_size, c_size, size_distract, input_range, eps_sigma):
  """Create a linear regression data set: X*w where x ~ N(0, 1), w ~ N(1,1) eps ~ N(0, eps_sigma)."""

  rng, new_rng, new_rng2, new_rng3, new_rng4 = jax.random.split(rng, 5)
  w = 1 + jax.random.normal(rng, shape=[i_size])
  x = jax.random.normal(new_rng, shape=[c_size, i_size])
  x_querry = jax.random.normal(new_rng2, shape=[1, i_size])

  y_data = jnp.squeeze(x@w) + jax.random.normal(new_rng3, shape=[c_size,])*eps_sigma

  y_target = x_querry@w + jax.random.normal(new_rng4, shape=[1,])*eps_sigma
  y_target = y_target[..., None]


  seq = jnp.concatenate([x, y_data[..., None]], -1)
  target = jnp.concatenate([x_querry, y_target], -1)
  x_querry_init = -1*x_querry.dot(jnp.ones_like(x_querry).T*0.0)
  zero = jnp.concatenate([x_querry, x_querry_init], -1)
  seq = jnp.concatenate([seq, zero], 0)
  return jnp.squeeze(seq), jnp.squeeze(target), w


@partial(jax.jit, static_argnums=(1, 2, 3))
def create_data_sin(rng, i_size, c_size, size_distract, input_range, eps_sigma):
  """Create a linear regression data set: X*w where x ~ N(0, 1), w ~ N(0,1) eps ~ N(0, eps_sigma)."""

  rng, new_rng, new_rng2, new_rng3, new_rng4, new_rng5, new_rng6 = jax.random.split(rng, 7)
  w = jax.random.normal(rng, shape=[i_size])

  v = jnp.cos(jnp.arange(1,i_size+1) * jnp.pi / 5)
  a = jax.random.normal(new_rng, shape=[c_size])
  av = jnp.broadcast_to(a, shape=(i_size, c_size)).transpose() * v

  u = jnp.sin(jnp.arange(1,i_size+1) * jnp.pi / 5)
  b = jax.random.normal(new_rng2, shape=[c_size])
  bu = jnp.broadcast_to(b, shape=(i_size, c_size)).transpose() * u

  eps = jax.random.normal(new_rng3, shape=[c_size, i_size]) * eps_sigma
  x = av + bu + eps

  a_q = jax.random.normal(new_rng4, shape=[1])
  b_q = jax.random.normal(new_rng5, shape=[1])
  eps_q = jax.random.normal(new_rng6, shape=[1, i_size]) * eps_sigma
  av_q = jnp.broadcast_to(a_q, shape=(i_size, 1)).transpose() * v
  bu_q = jnp.broadcast_to(b_q, shape=(i_size, 1)).transpose() * u

  x_querry = av_q + bu_q + eps_q


  y_data = jnp.squeeze(x@w)

  y_target = x_querry@w
  y_target = y_target[..., None]


  seq = jnp.concatenate([x, y_data[..., None]], -1)
  target = jnp.concatenate([x_querry, y_target], -1)
  x_querry_init = -1*x_querry.dot(jnp.ones_like(x_querry).T*0.0)
  zero = jnp.concatenate([x_querry, x_querry_init], -1)
  seq = jnp.concatenate([seq, zero], 0)
  return jnp.squeeze(seq), jnp.squeeze(target), w


In [6]:
# data creator
data_creator_normal = vmap(create_data_normal,
                      in_axes=(0, None, None, None, None, None), out_axes=0)

data_creator_noisy = vmap(create_data_normal_noisy,
                      in_axes=(0, None, None, None, None, None), out_axes=0)

data_creator_sine = vmap(create_data_sin,
                      in_axes=(0, None, None, None, None, None), out_axes=0)                      

In [12]:
#@title Training
results =list()
config.training_steps = 10000
for M in [5, 50,100]:  
  for ds_size in [5,10,15,20]:
    for e in [0, 0.1, 0.3, 0.5]:
      config.dataset_size = ds_size
      config.bs = M
      # interpolate GD and trained TF
      inter = True if (config.deq and not config.use_softmax and config.num_heads == 1) else False

      eval_rng = jax.random.PRNGKey(5)
      for cur_seed in range(0, config.num_seeds):
        config.seed = cur_seed
        optimiser, train_state, _, rng = init()
        rng, data_rng = jax.random.split(rng, 2)
        if config.analyse:
          lr_min=0.5

    
          params_gd = create_weights(config.input_size, 
                               1, 
                               config.dataset_size, 
                               lr_min,
                               jax.random.normal(data_rng, shape=[1, 1, config.input_size])*0 ,
                               lin_diag=False, 
                               gd_deq=config.gd_deq,
                               num_layers=config.num_layers,
                               input_mlp_rnd=rng if (config.input_mlp or config.in_proj) else None,
                               in_proj=config.in_proj)
          if config.num_layers > 1 or (config.in_proj and config.num_layers == 1):
            if cur_seed == 0:
              
              lr_min=0.5
              params_init = create_weights(config.input_size, 
                                     1, 
                                     config.dataset_size, lr_min,
                                    jax.random.normal(data_rng, shape=[1, 1, config.input_size])*0,
                                     lin_diag=True, 
                                     gd_deq=config.gd_deq,
                                     num_layers=config.num_layers,
                                    input_mlp_rnd=eval_rng if (config.input_mlp or config.in_proj) else None,
                                    in_proj=config.in_proj)
              
              params_gd_trained, data_rng = pre_train_gd_hps(eval_rng, params_init)
          else:
             params_gd_trained = params_gd

        eval_data = data_creator_sine(jax.random.split(eval_rng, num=1),
                                 config.input_size,
                                 config.dataset_size,
                                 config.size_distract,
                                 config.input_range,
                                 e)
        if config.analyse:
          loss_gd, _, _ = predict_test.apply(params_gd, eval_rng, eval_data,True)
          loss_gd_trained, _, _ = predict_test.apply(params_gd_trained, eval_rng,
                                                eval_data, True)    
        original_data_rng = data_rng   
        for step in range(config.training_steps):
          if config.cycle_data > 0:
            if step % config.cycle_data == 0:
              data_rng = original_data_rng

          rng, data_rng = jax.random.split(data_rng, 2)
          train_data = data_creator_sine(jax.random.split(rng, num=config.bs), 
                              config.input_size,
                              config.dataset_size,
                              config.size_distract,
                              config.input_range,
                              e)
          train_state, metrics = update(train_state, train_data, optimiser)
          
        loss_trans, _, _ = predict_test.apply(train_state.params, eval_rng,
                                            eval_data, False)
      results.append([M, ds_size, e, loss_gd.tolist(), loss_trans.tolist()])
          #loss_trans_list[cur_seed].append(loss_trans)
          #loss_trans_train_list[cur_seed].append(metrics['train_loss'].item(),)



In [13]:
import pandas as pd
results_df = pd.DataFrame(results, columns = ['M', 'ds_size', 'e', 'loss_gd', 'loss_trained'])
results_df.to_csv('results_lin_att_20240402_sine.csv', index = False)